In [15]:

import random
import pickle
import copy
import math
import os
import json
import time
import logging

import geopandas as gpd
import pandas as pd




In [48]:

vtd = gpd.read_file("./data/NC_VTD.shp").to_crs("EPSG:4269")



blocks = gpd.read_file("./data/tl_2016_37_tabblock10/tl_2016_37_tabblock10.shp").to_crs("EPSG:4269")

block_id = "GEOID10"
joined = gpd.sjoin(blocks, vtd, how = 'left',  predicate = 'intersects' )
mapping = joined[[block_id, "VTD"]]

#display(joined[[geo_id, "VTD"]].groupby(geo_id).agg('count').value_counts())

In [58]:
bg = gpd.read_file("./data/tl_2016_37_bg.shp").to_crs("EPSG:4269")

DriverError: ./data/tl_2016_37_bg.shp: No such file or directory

In [63]:
[x for x in joined.columns if "geo" in x]

['geometry']

In [ ]:
def impute_down(coarse_partition, coarse_id = "vtdid",
                 fine_partition, fine_id = "blockid"):
    # Given a function on coarse partition, imput down to fine partition based on overlap size

    
    joined = gpd.sjoin(coarse_partition, fine_partition, how = 'left',  predicate = 'intersects' )
    mapping = joined[[coarse_id, fine_id]]

    fine_with_map = fine_partition.merge(mapping)


    joined_with_geos  = fine_with_map.merge(coarse,
                                                        left_on = coarse_id, right_on = coarse_id,
                                                        suffixes = ('_fine', '_coarse'))



    fine_geos = gpd.GeoSeries(joined_with_geos.geometry_fine).to_crs("ESRI:103500")
    geo_with_vtd_shape_data['geo_intersect'] = geo_geos.intersection(gpd.GeoSeries(joined_with_geos.geometry_coarse).to_crs("ESRI:103500"))
    geo_with_vtd_shape_data['geo_area'] = geo_geos.area
    geo_with_vtd_shape_data['intersect_area'] = gpd.GeoSeries(geo_with_vtd_shape_data.geo_intersect).to_crs("ESRI:103500").area
    geo_with_vtd_shape_data['area_proportion'] =  geo_with_vtd_shape_data['intersect_area'] / geo_with_vtd_shape_data['geo_area']


    geo_with_vtd_shape_data = geo_with_vtd_shape_data[geo_with_vtd_shape_data.area_proportion >= .99]

    

    



In [49]:
# Intersect geos with VTDs, compute area ratios

# add back in block shape data
geo_table_with_shape_data = blocks.merge(mapping)
# add back in VTD shape data
geo_with_vtd_shape_data  = geo_table_with_shape_data.merge(vtd,
                                                      left_on = "VTD", right_on = "VTD",
                                                      suffixes = ('_block', '_vtd'))



geo_geos = gpd.GeoSeries(geo_with_vtd_shape_data.geometry_block).to_crs("ESRI:103500")
geo_with_vtd_shape_data['geo_intersect'] = geo_geos.intersection(gpd.GeoSeries(geo_with_vtd_shape_data.geometry_vtd).to_crs("ESRI:103500"))
geo_with_vtd_shape_data['geo_area'] = geo_geos.area
geo_with_vtd_shape_data['intersect_area'] = gpd.GeoSeries(geo_with_vtd_shape_data.geo_intersect).to_crs("ESRI:103500").area
geo_with_vtd_shape_data['area_proportion'] =  geo_with_vtd_shape_data['intersect_area'] / geo_with_vtd_shape_data['geo_area']


geo_with_vtd_shape_data = geo_with_vtd_shape_data[geo_with_vtd_shape_data.area_proportion >= .99]

In [50]:
pops = pd.read_csv('./data/us2016.csv')
pops['block_fips'] = pops.block_fips.astype("str")
vote_fields = ["EL16G_GV_R" , "EL16G_GV_D", "TOTPOP"]
fields = ['VTD', "GEOID10", 'geometry', 'area_proportion' ] +  vote_fields
geos_simple = geo_with_vtd_shape_data.rename(columns = {"geometry_block" : "geometry" })[fields]
geo_with_pop = geos_simple.merge(pops, left_on = "GEOID10", right_on = 'block_fips',)

In [51]:
# VTD level sums -- doing because the VTD tot pop is generally not equal to the sum of the block pops in it
# This indicates some data level issue , probably in addition to the non exact containment issue.
# Code to check:
# merged = geo_with_pop[["TOTPOP", "VTD"]].merge(geo_with_pop[["pop2016", "VTD"]].groupby("VTD").agg(sum).reset_index(), left_on = "VTD", right_on = "VTD")
# (merged.TOTPOP / merged.pop2016).sort_values().apply(lambda x : round(x,1) ).value_counts()
# But ignoring for now.
vtds_block_pops = geo_with_pop[["pop2016", "VTD"]].groupby("VTD").agg(sum)
vtds_block_pops.columns = ["pop2016_VTD_total"]
geo_with_pop = geo_with_pop.merge(vtds_block_pops, left_on = "VTD", right_index = True)
geo_with_pop['pop_proportion'] = geo_with_pop.pop2016 /  geo_with_pop.pop2016_VTD_total

# Filter out and set to zero for the ones where the intersectiokn is too small
# NB: I did not include this step the first itme running it; TODO -- rerun.

vote_fields = ["EL16G_GV_R" , "EL16G_GV_D", "TOTPOP"]
for field in vote_fields:
    # Eventually replace with ecological inference or something
    geo_with_pop[field + "_block"] = geo_with_pop['pop_proportion'] * geo_with_pop[field]

gdf_dedupped = geo_with_pop.sort_values('area_proportion').drop_duplicates('GEOID10', keep='last')
gdf_dedupped.pop2016.sum() / geo_with_pop.pop2016.sum()

gpd.GeoDataFrame(gdf_dedupped).to_file('./processed_data_outputs/blocks_with_votes.shp')

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [53]:

gpd.GeoDataFrame(gdf_dedupped).to_file('./processed_data_outputs/blocks_with_votes.shp')

/var/folders/wm/q6gwq3q56yj8rdyj_h_qhkd40000gn/T/ipykernel_5668/1556453969.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(gdf_dedupped).to_file('./processed_data_outputs/blocks_with_votes.shp')


0.6278388760008075

/var/folders/wm/q6gwq3q56yj8rdyj_h_qhkd40000gn/T/ipykernel_5668/2074470958.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  test.to_file('./processed_data_outputs/blocks_with_votes.shp')


In [54]:
gdf_subset = gdf_dedupped[['VTD', "GEOID10", 'geometry', "TOTPOP", "pop2016", "pop_proportion", 'area_proportion' ] +  [ x + "_block" for x in vote_fields] ]
gdf_subset = gpd.GeoDataFrame(gdf_subset)

test = gpd.GeoDataFrame(gdf_subset, geometry="geometry", crs= "ESRI:103500")#.head(10)
test.to_file('./processed_data_outputs/blocks_with_votes.shp')

/var/folders/wm/q6gwq3q56yj8rdyj_h_qhkd40000gn/T/ipykernel_5668/2014793804.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  test.to_file('./processed_data_outputs/blocks_with_votes.shp')


In [55]:
test

,VTD,GEOID10,geometry,TOTPOP,pop2016,pop_proportion,area_proportion,EL16G_GV_R_block,EL16G_GV_D_block,TOTPOP_block
212961,37087FC-1,370879201021102,"POLYGON ((-83.098 35.776, -83.098 35.776, -83....",1020,0,0.000000,0.990000,0.000000,0.000000,0.000000
259347,37087BE-4,370879202001071,"POLYGON ((-82.808 35.574, -82.808 35.574, -82....",2303,326,0.139555,0.990001,92.803938,59.310788,321.394692
200308,3719101,371910001013036,"POLYGON ((-78.084 35.549, -78.084 35.550, -78....",3905,75,0.019773,0.990009,20.702610,10.064593,77.214606
182673,37119127,371190064042067,"POLYGON ((-80.790 35.468, -80.790 35.468, -80....",4976,225,0.038213,0.990010,68.019701,66.376529,190.149457
232969,37027PR07,370270309001058,"POLYGON ((-81.779 36.004, -81.779 36.004, -81....",2754,0,0.000000,0.990012,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
92042,37125NSP,371259509003023,"POLYGON ((-79.392 35.174, -79.391 35.175, -79....",3081,3,0.000901,1.000000,0.747296,0.753606,2.777344
224,3703742,370370208003016,"POLYGON ((-79.107 35.731, -79.107 35.731, -79....",2042,0,0.000000,1.000000,0.000000,0.000000,0.000000
59420,371471201,371470009003065,"POLYGON ((-77.363 35.659, -77.363 35.660, -77....",7896,2,0.000244,1.000000,0.303371,0.292379,1.928676
58880,371471504,371470006011076,"POLYGON ((-77.425 35.598, -77.424 35.598, -77....",9389,1,0.000101,1.000000,0.104822,0.359951,0.949055


In [ ]:
test_in = gpd.read_file('./processed_data_outputs/blocks_with_votes.shp')

: 

: 

In [29]:

from gerrychain import Graph

gdf_subset['pos'] = gdf_subset.centroid
gdf_subset = gdf_subset.rename(columns = { "EL16G_GV_D_block" : "dem", "EL16G_GV_R_block" : "rep"})
gdf_subset['dem_prop'] = gdf_subset.dem / gdf_subset.TOTPOP_block
gdf_subset['rep_prop'] = gdf_subset.rep / gdf_subset.TOTPOP_block

blocks_graph = Graph.from_geodataframe(gdf_subset, ignore_errors=True)
blocks_graph.to_json('./processed_data_outputs/blocks_graph.json', include_geometries_as_geojson=True)

/Users/elle/opt/anaconda3/envs/gerry/lib/python3.9/site-packages/gerrychain/graph/adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)
/Users/elle/opt/anaconda3/envs/gerry/lib/python3.9/site-packages/gerrychain/graph/graph.py:272: UserWarning: NA values found in column dem_prop!
  warnings.warn("NA values found in column {}!".format(column))
/Users/elle/opt/anaconda3/envs/gerry/lib/python3.9/site-packages/gerrychain/graph/graph.py:272: UserWarning: NA values found in column rep_prop!
  warnings.warn("NA values found in column {}!".format(column))


In [31]:
blocks_graph

<Graph [288987 nodes, 708356 edges]>

In [33]:
def condensce_by_partition(graph, part):
    # given graph, and label field for each node, return graph where all the nodes with the same lable have been contracted
    # also, add up the pops and votes in each block
    graph_copy = graph.copy()
    
    for supernode in part.parts.values():
        nodes = list(supernode)
        for node in nodes[1:]:
            
            tot_pop = graph_copy.nodes[nodes[0]]['TOTPOP_block'] + graph_copy.nodes[node]['TOTPOP_block']
            tot_dem = graph_copy.nodes[nodes[0]]['dem']+ graph_copy.nodes[node]['dem']
            tot_rep = graph_copy.nodes[nodes[0]]['rep'] + graph_copy.nodes[node]['rep']
            nx.contracted_nodes(graph_copy, nodes[0], node, copy = False)
            graph_copy.nodes[nodes[0]]['TOTPOP_block'] = tot_pop
            graph_copy.nodes[nodes[0]]['dem'] = tot_dem
            graph_copy.nodes[nodes[0]]['rep'] = tot_rep
            
    return graph_copy

In [57]:
gdf_dedupped.columns

Index(['VTD', 'GEOID10', 'geometry', 'area_proportion', 'EL16G_GV_R',
       'EL16G_GV_D', 'TOTPOP', 'stateabbr', 'block_fips', 'hu2010', 'hu2011',
       'hu2012', 'hu2013', 'hu2014', 'hu2015', 'hu2016', 'pop2010', 'pop2011',
       'pop2012', 'pop2013', 'pop2014', 'pop2015', 'pop2016', 'hh2010',
       'hh2011', 'hh2012', 'hh2013', 'hh2014', 'hh2015', 'hh2016',
       'pop2016_VTD_total', 'pop_proportion', 'EL16G_GV_R_block',
       'EL16G_GV_D_block', 'TOTPOP_block'],
      dtype='object')

In [32]:
#blocks_graph_in = Graph.from_json('./processed_data_outputs/blocks_graph.json')

In [ ]:
import pandas as pd
import pymc3 as pm

# # read in data
# county_votes = pd.read_csv('county_votes.csv')
# block_demographics = pd.read_csv('block_demographics.csv')

# # merge dataframes
# merged_data = pd.merge(county_votes, block_demographics, on='county')

block_demographics = [""]

# create model with pymc3
with pm.Model() as model:
    # priors for county-level votes
    county_vote_mean = pm.Normal('vtd_dem_prop_mean', mu=0, sd=10)
    county_vote_sd = pm.HalfNormal('vtd_dem_prop_mean', sd=10)
    
    # incorporate demographic features into model
    for feature in block_demographics.columns:
        if feature != 'votes':
            feature_coef = pm.Normal(f'{feature}_coef', mu=0, sd=1)
            county_vote_mean += feature_coef * merged_data[feature]
    
    # likelihood of block-level votes
    block_votes = pm.Normal('block_votes', mu=county_vote_mean, sd=county_vote_sd, observed=merged_data['votes'])
    
    # fit model
    trace = pm.sample(1000)

# summarize results
pm.summary(trace)


: 

: 

: 